In [1]:
import MDAnalysis as md
import os
import numpy as np
import nglview as ng
import re
import subprocess

In [26]:
def get_dict_types(sel):
    '''Given an atom selection from MDAnalysis, 
    it will output a dictionary contaning all the beadtypes
    with their corresponding vdw radii.'''
    D = {}
    for idx, i in enumerate(sel.types):
        name = sel.names[idx]
        if re.match('^T', i):
            radii = 0.191
        elif re.match('^S', i):
            radii = 0.230
        else:
            radii = 0.264
        D[i] = radii
    return D

def get_dict_names(sel, writefile=False):
    '''Given an atom selection from MDAnalysis, 
    it will output a dictionary contaning all the unique beadnames
    with their corresponding vdw radii.
    NB! This is not great, if the same beadname is used with different
    beadtypes, hence different vdw radii'''
    D = {}
    for idx, i in enumerate(sel.types):
        name = sel.names[idx]
        if re.match('^T', i):
            radii = 0.191
        elif re.match('^S', i):
            radii = 0.230
        else:
            radii = 0.264
        D[name] = radii
    if writefile == True:
        sasa = open('vdwradii.dat', 'w')
        sasa.write(';CG van der walls radii\n')
        for k in D.keys():
            sasa.write('???  {}    {}\n'.format(k, D[k]))
    return D


def run_SASA(resolution, target, sel, dir_out):
    ''''Function for doing SASA calculation with gromacs. 
    Provide the resolution AA og CG, the target gro and xtc files as a list, the selection sel, and the working directory dir_out'''
    try:
        dir_writing = '{}/{}'.format(dir_out, resolution)
        os.mkdir(dir_writing)
    except FileExistsError:
        pass
    gro = target[0]
    xtc = target[1]
    u = md.Universe(gro, xtc)

    ## Create index file and spit out a gro
    ## Copy vdwradii to target directory.
    if resolution == 'AA':
        tgt = u.select_atoms('{}'.format(sel)).atoms.select_atoms('all')
        subprocess.call('cp /data1/lisbeth/Params/Pzifer_alcoholamine/7_SASA_longer_bonds/vdwradii_AllAtom_rowland.dat {}/vdwradii.dat'.format(dir_writing)
                    , shell=True)
    else:
        tgt = u.select_atoms('{}'.format(sel)).atoms.select_atoms('all')
        get_dict_names(tgt, writefile=True)

    tgt.write("{}/index.ndx".format(dir_writing), mode="w", name= 'TGT')
    tgt.atoms.write(f"{dir_writing}/gro.gro")

    ## Calculate SASA & connoly surface
    os.chdir(dir_writing)
    print ('entering {}'.format(dir_writing))
    p = subprocess.Popen(f"gmx sasa -f {xtc} -s {gro} -n {dir_writing}/index.ndx -ndots 4800 -probe 0.191 -or {dir_writing}/resarea_SASA.xvg -o {dir_writing}/SASA.xvg "
                , stdin=subprocess.PIPE, shell=True, universal_newlines=True)
    p.communicate('TGT')
    p.wait()

    os.chdir('../')
    return


In [ ]:
sele = 'segid seg_0_A' #MDAnalysis string selection -> writes out the needed index file for gromacs to perform the analysis

In [44]:
main_dir = '/data1/lisbeth/SASA_Proj'
os.chdir(main_dir)

In [45]:
CG = ['{}/SPC_L11_CG_2.tpr'.format(main_dir), '{}/SPC_L11_CG_1_whole_skip100.xtc'.format(main_dir)]

In [46]:
## Running the gromacs commands
run_SASA('CG', CG, 'sele, main_dir) #Do it for chain A of the protein. If done for everything, gromacs only takes the unique residues and provide an average across

entering /data1/lisbeth/SASA_Proj/CG

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary.

NOTE: From version 5.0 gmx sasa uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. Bondi
van der Waals Volumes and Radii
J. Phys. Chem. 68 (1964) pp. 441-451
-------- -------- --- Thank You --- -------- --------



                       :-) GROMACS - gmx sasa, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
      

In [47]:
#Load the output from gromacs into an array
sasa_res = np.loadtxt('CG/resarea_SASA.xvg', comments=('#','@')) #3 coulumns, resid, average area, standard deviation all in nm^2 

In [90]:
resids = sasa_res[:,0]

In [91]:
#convert into a dictinoary 
D = {}
for idx, res in enumerate(resids):
    D[res] = [sasa_res[idx,1], sasa_res[idx,2]] 

In [93]:
D

{1.0: [1.472, 0.252],
 2.0: [0.964, 0.199],
 3.0: [0.832, 0.095],
 4.0: [1.011, 0.085],
 5.0: [0.694, 0.219],
 6.0: [1.362, 0.354],
 7.0: [0.854, 0.08],
 8.0: [0.728, 0.083],
 9.0: [0.421, 0.15],
 10.0: [0.636, 0.106],
 11.0: [1.895, 0.126],
 12.0: [1.335, 0.132],
 13.0: [0.548, 0.122],
 14.0: [0.856, 0.138],
 15.0: [1.461, 0.133],
 16.0: [1.407, 0.171],
 17.0: [0.462, 0.235],
 18.0: [0.577, 0.108],
 19.0: [1.363, 0.211],
 20.0: [1.405, 0.229],
 21.0: [0.44, 0.126],
 22.0: [0.792, 0.093],
 23.0: [0.881, 0.122],
 24.0: [0.816, 0.259],
 25.0: [1.303, 0.251],
 26.0: [0.229, 0.037],
 27.0: [1.113, 0.125],
 28.0: [0.702, 0.198],
 29.0: [0.487, 0.086],
 30.0: [0.834, 0.066],
 31.0: [0.896, 0.091],
 32.0: [0.68, 0.091],
 33.0: [0.538, 0.09],
 34.0: [1.148, 0.084],
 35.0: [1.077, 0.08],
 36.0: [1.936, 0.16],
 37.0: [1.168, 0.095],
 38.0: [0.264, 0.037],
 39.0: [0.389, 0.126],
 40.0: [0.672, 0.126],
 41.0: [1.077, 0.062],
 42.0: [1.183, 0.089],
 43.0: [0.152, 0.072],
 44.0: [0.33, 0.059],
 45.0